Installing Packages

In [ ]:

!pip install requests 

!pip install BeautifulSoup4 

!pip install scrapy 

!pip install nltk 

!pip install xgboost 

!pip install pandastable 

!pip install gensim 


#requests: HTTP library
#BeautifulSoup4: web scraping library
#scrapy: Web Scraping framework
#nltk: Natural Language Processing toolkit
#xgboost: Gradient boosting library
#pandastable: Data Structure and analysis tool
#gensim: document manipulation library



In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import datetime
import string
import json
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
from nltk import pos_tag
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\kccra\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\kccra\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kccra\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kccra\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# Seed Page
URL = "https://pureportal.coventry.ac.uk/en/organisations/coventry-university/persons/"

#Profile url allowed to be crawled as specified in robots.txt page
url_profile = "https://pureportal.coventry.ac.uk/en/persons/"

Defining a function to get total number of pages consisting profile detalis

In [3]:
def get_totoal_number_of_pages():
    first_page = requests.get(URL)
    soup = BeautifulSoup(first_page.text, 'html.parser')
    final_page = soup.select('#main-content > div > section > nav > ul > li:nth-child(12) > a')[0]['href']
    n = final_page.split('=')[-1]
    return int(n)   
number_of_pages = get_totoal_number_of_pages() 

In [4]:
number_of_pages

46

Thus, there are total of 46 pages

Defining a method to creawl researchers who have research publications and are part of the "School of Computing, Electronics and Maths"

In [11]:
def find_department(researcher):
    
    l1 = researcher.find('div', class_='rendering_person_short')
      
    for span in l1.find_all('span'):
        # find department
        if span.text == str('School of Computing, Mathematics and Data Sciences'):
            name = researcher.find('h3', class_='title').find('span').text
            return name
        else:
            pass

def create_csv():
    database = pd.DataFrame(columns=['Title', 'Author', 'Published', 'Link'])
    database.to_csv('db.csv')
    
def update_csv(database):
    current_data = pd.read_csv(database, index_col="Unnamed: 0")
    return current_data        

def enter_each_researchers_publication(researcher, url, df):
    new_url = url + str(researcher).replace(' ','-').lower() + '/publications/'
    page = requests.get(new_url)
    soup = BeautifulSoup(page.content, "html.parser")
    results = soup.find(id="main-content")
    papers = results.find_all("li", class_="list-result-item")
    
    if papers is not None:
        for paper in papers:
            if paper is not None:

                t1 = paper.find('h3', class_='title')
                if t1 is not None:
                    title=t1.find('span')

                a1 = paper.find('a', class_='link person')
                if a1 is not None:
                    author = a1.find('span')

                date = paper.find('span', class_="date")
                if date is not None:
                    d_txt= date.text

                l1 = paper.find('h3', class_='title')
                if l1 is not None:
                    link = l1.find('a', href=True)['href']

                opening = pd.read_csv('db.csv', index_col="Unnamed: 0")
                d=[{'Title': title.text,'Author': author.text,'Published': d_txt,'Link': link}]
                opening1 = pd.DataFrame(d, index=list(range(len(d))))
                opening =pd.concat([opening,opening1])
                opening.to_csv('db.csv')
        

Function to carryout Scraping

In [12]:
## Scrape function
def scrape(n_of_pages):
    df = update_csv('db.csv')
    i=0
    while True:
    
        if i > 47:
            break
            
        if i>0:
            url = URL + '?page=' + str(i)
        else:
            url = URL
    
        i = i+1
        # scraping starts here
        page = requests.get(url)
        soup = BeautifulSoup(page.content, "html.parser")
        results = soup.find(id="main-content")
        researchers = results.find_all("li", class_="grid-result-item")

        for researcher in researchers:
            # Check if researcher has any papers
            check = researcher.find('div', class_='stacked-trend-widget')
            if check:
                name = find_department(researcher)
                if name is None:
                    pass
                else:
                    enter_each_researchers_publication(name, url_profile, df)

In [13]:
create_csv()

%time scrape(number_of_pages)  

Wall time: 4min 43s


In [9]:
db_hf=pd.read_csv("db.csv")

In [10]:
db_hf.head()

,Unnamed: 0,Title,Author,Published,Link
333,0,Using Machine Learning for Anomaly Detection o...,"Kasap, S.",Nov 2022,https://pureportal.coventry.ac.uk/en/publicati...
564,0,DSP-based bottleneck bandwidth estimation,"Wang, X.",2009,https://pureportal.coventry.ac.uk/en/publicati...
10,0,An Active Attack on Token-Based Security Protocol,"Abdelshafy, M.",Apr 2001,https://pureportal.coventry.ac.uk/en/publicati...
3,0,Resisting blackhole attacks on MANETs,"Abdelshafy, M.",Jan 2016,https://pureportal.coventry.ac.uk/en/publicati...
417,0,A framework for inter-society communication in...,"Mitchell, F.",2002,https://pureportal.coventry.ac.uk/en/publicati...
450,0,Board structure and corporate R&D intensity: ...,"Moustafa, M.",18 Jun 2020,https://pureportal.coventry.ac.uk/en/publicati...
534,0,In at the deep end: an activity-led introducti...,"Halloran, J.",Sept 2012,https://pureportal.coventry.ac.uk/en/publicati...
382,0,Extraction of cardiac and respiratory motion i...,"Ma, Y.",2013,https://pureportal.coventry.ac.uk/en/publicati...
131,0,Early feeding and growth pattern in infants: U...,"Daneshkhah, A.",2018,https://pureportal.coventry.ac.uk/en/publicati...
339,0,Acceleration of Polynomial Matrix Multiplicati...,"Kasap, S.",Sept 2019,https://pureportal.coventry.ac.uk/en/publicati...


In [12]:
del db_hf['Unnamed: 0']

In [13]:
start=1
db_hf.insert(0, 'SN', range(start, start + db_hf.shape[0]))

In [14]:
db_hf.sample(7)

,SN,Title,Author,Published,Link
418,419,An introduction to knowledge acquisition,"Mitchell, F.",1998,https://pureportal.coventry.ac.uk/en/publicati...
60,61,Constructing gene regulatory networks from mic...,"Chatrabgoun, O.",24 Jul 2020,https://pureportal.coventry.ac.uk/en/publicati...
542,543,Horizon Visibility Graphs and Time Series Merg...,"Stephen, C.",20 Jun 2019,https://pureportal.coventry.ac.uk/en/publicati...
494,495,"Integration, optimization and usability of ent...","Shah, N.",2013,https://pureportal.coventry.ac.uk/en/publicati...
40,41,Implementation of a proactive load sharing scheme,"Anane, R.",2003,https://pureportal.coventry.ac.uk/en/publicati...
152,153,A game-based learning approach to road safety:...,"Dunwell, I.",26 Apr 2014,https://pureportal.coventry.ac.uk/en/publicati...
121,122,The prevalence of Restless Legs Syndrome/Willi...,"Daneshkhah, A.",13 Apr 2020,https://pureportal.coventry.ac.uk/en/publicati...


In [15]:
Scrapped_hf=pd.read_csv("db.csv")

In [19]:
Scrapped_hf

,SN,Title,Author,Published,Link
0,1,Cross-Layer Multipath Multichannel MAC protoco...,"Abdelshafy, M.",Jun 2018,https://pureportal.coventry.ac.uk/en/publicati...
1,2,Performance evaluation of Receiver Directed Tr...,"Abdelshafy, M.",4 May 2017,https://pureportal.coventry.ac.uk/en/publicati...
2,3,Reliable Multipath Multi-channel Route Migrati...,"Abdelshafy, M.",23 Nov 2017,https://pureportal.coventry.ac.uk/en/publicati...
3,4,Resisting blackhole attacks on MANETs,"Abdelshafy, M.",Jan 2016,https://pureportal.coventry.ac.uk/en/publicati...
4,5,Dynamic Source Routing under Attacks,"Abdelshafy, M.",12 Oct 2015,https://pureportal.coventry.ac.uk/en/publicati...
...,...,...,...,...,...
569,570,Extracting partition function zeros from Fukui...,"Holovatch, Y.",18 Oct 2021,https://pureportal.coventry.ac.uk/en/publicati...
570,571,"The large-m limit, and spin liquid correlation...","Yavors'kii, T.",2017,https://pureportal.coventry.ac.uk/en/publicati...
571,572,Refrustration and competing orders in the prot...,"Yavorskyi, T.",7 Jan 2016,https://pureportal.coventry.ac.uk/en/publicati...
572,573,Optimized GPU simulation of continuous-spin gl...,"Yavors'kii, T.",2012,https://pureportal.coventry.ac.uk/en/publicati...


In [17]:
del Scrapped_hf['Unnamed: 0']

In [18]:
start=1
Scrapped_hf.insert(0, 'SN', range(start, start + Scrapped_hf.shape[0]))

In [28]:
a_row = Scrapped_hf.loc[5,:].copy()
a_row

SN                                                           6
Title        AODV & SAODV under Attack: Performance Comparison
Author                                          Abdelshafy, M.
Published                                             Jun 2014
Link         https://pureportal.coventry.ac.uk/en/publicati...
Name: 5, dtype: object

# Text Preprocessing

In [23]:
# Removing stop words
sw = stopwords.words("english")
lemmatizer = WordNetLemmatizer()


def tp1(txt):
    # converstion to lowercase
    txt = txt.lower()   
    # Removig punctuation marks
    txt = txt.translate(str.maketrans('','',string.punctuation))   
    txt = lematize(txt)
    return txt


def fwpt(word):
    tag = pos_tag([word])[0][1][0].upper()
    hash_tag = {"V": wordnet.VERB, "R": wordnet.ADV,"N": wordnet.NOUN,"J": wordnet.ADJ}         
    return hash_tag.get(tag, wordnet.NOUN)

def lematize(text):
        tkns = nltk.word_tokenize(text)
        ax = ""
        for each in tkns:
            if each not in sw:
                ax += lemmatizer.lemmatize(each, fwpt(each)) + " "
        return ax

In [29]:
# A Sample title
a_row['Title']

'AODV & SAODV under Attack: Performance Comparison'

In [30]:
# calling  lowercase and punctuation removal method
tp1(a_row['Title'])

'aodv saodv attack performance comparison '

In [31]:
# calling lematization method
lematize(tp1(a_row['Title']))

'aodv saodv attack performance comparison '

### Processing all the dataframes

In [33]:
processed_db = Scrapped_hf.copy()

def preprocess_df(df):
    df.Title = df.Title.apply(tp1)
    df.Author = df.Author.str.lower()
    df = df.drop(columns=['Author','Published'], axis=1)
    return df
    
preprocess_df(processed_db)
processed_db.head(10)

,SN,Title,Author,Published,Link
0,1,crosslayer multipath multichannel mac protocol...,"abdelshafy, m.",Jun 2018,https://pureportal.coventry.ac.uk/en/publicati...
1,2,performance evaluation receiver direct transmi...,"abdelshafy, m.",4 May 2017,https://pureportal.coventry.ac.uk/en/publicati...
2,3,reliable multipath multichannel route migratio...,"abdelshafy, m.",23 Nov 2017,https://pureportal.coventry.ac.uk/en/publicati...
3,4,resist blackhole attack manet,"abdelshafy, m.",Jan 2016,https://pureportal.coventry.ac.uk/en/publicati...
4,5,dynamic source rout attack,"abdelshafy, m.",12 Oct 2015,https://pureportal.coventry.ac.uk/en/publicati...
5,6,aodv saodv attack performance comparison,"abdelshafy, m.",Jun 2014,https://pureportal.coventry.ac.uk/en/publicati...
6,7,aodv rout protocol performance analysis manet ...,"abdelshafy, m.",Jun 2014,https://pureportal.coventry.ac.uk/en/publicati...
7,8,resist flood attack aodv,"abdelshafy, m.",Nov 2014,https://pureportal.coventry.ac.uk/en/publicati...
8,9,analysis security attack aodv rout,"abdelshafy, m.",Dec 2013,https://pureportal.coventry.ac.uk/en/publicati...
9,10,analysis evaluation wireless lan security prot...,"abdelshafy, m.",Nov 2002,https://pureportal.coventry.ac.uk/en/publicati...


### Creating index

In [34]:
## Indexing Function
def apply_index(inputs, index):
    words = inputs.Title.split()
    SN = int(inputs.SN)
    for word in words:
        if word in index.keys():
            if SN not in index[word]:
                index[word].append(SN)
        else:
            index[word] = [SN]
    return index

def full_index(df, index):
    for x in range(len(df)):
        inpt = df.loc[x,:]
        ind = apply_index(inputs=inpt, index=index)
    return ind

def construct_index(df, index):
    queue = preprocess_df(df)
    ind = full_index(df=queue, index=index)
    return ind

indexed = full_index(processed_db, 
                     index = {})


indexes = construct_index(df=Scrapped_hf, 
                          index = {})

In [35]:
indexed

{'crosslayer': [1],
 'multipath': [1, 3, 552],
 'multichannel': [1, 3, 93, 477],
 'mac': [1],
 'protocol': [1, 2, 7, 10, 11, 12, 26, 337, 404],
 'manet': [1, 2, 4, 7],
 'performance': [2,
  6,
  7,
  16,
  102,
  128,
  266,
  309,
  351,
  352,
  353,
  354,
  355,
  356,
  358,
  532,
  546,
  549,
  556,
  558,
  566,
  567,
  568],
 'evaluation': [2,
  10,
  17,
  18,
  20,
  72,
  84,
  105,
  128,
  154,
  183,
  228,
  275,
  305,
  370,
  377,
  394,
  400,
  483,
  510,
  514,
  532,
  533],
 'receiver': [2],
 'direct': [2],
 'transmission': [2, 221, 264],
 'single': [2, 432],
 'transceiver': [2],
 'reliable': [3],
 'route': [3, 552],
 'migration': [3],
 'multi': [3],
 'linkfailure': [3],
 'wireless': [3,
  10,
  219,
  220,
  221,
  264,
  266,
  267,
  268,
  269,
  270,
  319,
  446,
  485,
  537,
  538,
  539,
  540,
  546,
  554,
  560],
 'ad': [3, 552, 562],
 'hoc': [3, 552, 562],
 'network': [3,
  24,
  61,
  69,
  93,
  105,
  106,
  114,
  115,
  118,
  129,
  192,
  

In [36]:
with open('indexes.json', 'w') as new_f:
    json.dump(indexes, new_f, sort_keys=True, indent=4)
    
with open('indexes.json', 'r') as file:
    data = json.load(file)

def index_2(df, x_path):
    if len(df) > 0:
        with open(x_path, 'r') as file:
            prior_index = json.load(file)
        new_index = construct_index(df = df, index = prior_index)
        with open(x_path, 'w') as new_f:
            json.dump(new_index, new_f, sort_keys=True, indent=4)

In [37]:
data


{'1': [95, 96],
 '15360bit': [53],
 '2000': [451],
 '2009': [324],
 '2013': [47],
 '2015': [286],
 '2d': [56, 97, 381, 387],
 '2nd': [556, 564],
 '30': [183],
 '3d': [365, 377, 381, 387, 396, 408, 409],
 '40': [211, 476],
 '4f': [134],
 '4th': [324],
 '80211': [549, 566, 568],
 'abcsg': [246],
 'ability': [207, 475],
 'ablation': [402],
 'absorption': [51],
 'abstract': [414],
 'accelerate': [574],
 'acceleration': [340],
 'accelerator': [341],
 'access': [31, 550],
 'accommodation': [136, 149],
 'accord': [282],
 'accuracy': [335],
 'accurate': [60, 112, 432],
 'achievementbased': [179],
 'acquire': [470],
 'acquisition': [419, 421, 423, 424],
 'action': [84, 199, 321],
 'active': [11, 47, 281, 557],
 'activity': [49, 50, 157, 192, 216, 297, 491],
 'activityled': [320, 535],
 'acute': [391, 398],
 'ad': [3, 552, 562],
 'adapt': [282, 291, 295],
 'adaptable': [490],
 'adaptation': [17, 21, 519],
 'adaptive': [20, 140, 166, 173, 174, 228, 275, 287, 559],
 'adaptively': [430],
 'adaptivi

# Query Processor

In [38]:
#Split query into individual terms
def split_query(terms):
    each = tp1(terms)
    return each.split()


In [39]:
def union(lists):
    union = list(set.union(*map(set, lists)))
    union.sort()
    return union

def intersection(lists):
    intersect = list(set.intersection(*map(set, lists)))
    intersect.sort()
    return intersect

In [40]:
def vertical_search_engine(df, query, index=indexes):
    query_split = split_query(query)
    retrieved = []
    for word in query_split:
        if word in index.keys():
            retrieved.append(index[word])
            
            
    # Ranked Retrieval
    if len(retrieved)>0:
        high_rank_result = intersection(retrieved)
        low_rank_result = union(retrieved) 
        c = [x for x in low_rank_result if x not in high_rank_result]      
        high_rank_result.extend(c)
        result = high_rank_result
        
        final_output = df[df.SN.isin(result)].reset_index(drop=True)
    
        # Return result in order of Intersection ----> Union
        dummy = pd.Series(result, name = 'SN').to_frame()
        result = pd.merge(dummy, final_output, on='SN', how = 'left')
        
    else:
        result = 'No result found'
    
    return result

In [47]:
def test_search_engine():
    xtest = Scrapped_hf.copy()
    query = input("Enter your search query: ")
    return vertical_search_engine(xtest, query, indexed)
    
test_search_engine()


Enter your search query: Network Security


,SN,Title,Author,Published,Link
0,3,reliable multipath multichannel route migratio...,"abdelshafy, m.",23 Nov 2017,https://pureportal.coventry.ac.uk/en/publicati...
1,9,analysis security attack aodv rout,"abdelshafy, m.",Dec 2013,https://pureportal.coventry.ac.uk/en/publicati...
2,10,analysis evaluation wireless lan security prot...,"abdelshafy, m.",Nov 2002,https://pureportal.coventry.ac.uk/en/publicati...
3,11,active attack tokenbased security protocol,"abdelshafy, m.",Apr 2001,https://pureportal.coventry.ac.uk/en/publicati...
4,12,new attack azizdiffie security protocol,"abdelshafy, m.",Mar 2001,https://pureportal.coventry.ac.uk/en/publicati...
5,22,impact learn style adaptivity teach computer s...,"anane, r.",2015,https://pureportal.coventry.ac.uk/en/publicati...
6,24,social network enhancement nonformal learn,"anane, r.",2014,https://pureportal.coventry.ac.uk/en/publicati...
7,61,construct gene regulatory network microarray d...,"chatrabgoun, o.",24 Jul 2020,https://pureportal.coventry.ac.uk/en/publicati...
8,69,approximate nongaussian bayesian network use m...,"chatrabgoun, o.",Jan 2018,https://pureportal.coventry.ac.uk/en/publicati...
9,93,develop graph convolutional network mutual inf...,"daneshkhah, a.",28 Aug 2022,https://pureportal.coventry.ac.uk/en/publicati...


# Schedule Crawler for every 15 Days

In [65]:
days = 0
interval = 15
while days <= 1:
    scrape(number_of_pages)
    print(f"Crawled at {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f'Next crawl scheduled after {interval} days')
    time.sleep(interval)
    days = days + 1

Crawled at 2023-06-01 18:39:50
Next crawl scheduled after 15 days
Crawled at 2023-06-01 18:40:05
Next crawl scheduled after 15 days


# Generating  Graphical User Interface

In [51]:
from tkinter import *
from PIL import Image, ImageTk 
from tkinter import scrolledtext
from pandastable import Table, TableModel
from contextlib import suppress
import warnings
import tkinter as tk
warnings.filterwarnings('ignore')


image1 = Image.open('earth.png')
resized_image1 = image1.resize((300,100), Image.ANTIALIAS)

def new_gui(pic):
    window = Tk()
    window.title("Your Information retrieval Search Engine")
    window.geometry('1000x500')
    
    lbl = Label(window, text="Your Personal Search Engine", font=("Arial Bold", 30), padx=5, pady=5)
    lbl.grid(column=1, row=0)
    
    lbl2 = Label(window, text="Please Enter your search term: ", font=("Arial", 15), padx=5, pady=5)
    lbl2.grid(column=0, row=1)
    
    
    img = ImageTk.PhotoImage(pic)
    
    lbl3 = Label(image=img)
    lbl3.image = img
    lbl3.grid(column=1, row=3, padx=5, pady=5)
    
    
    
    query = Entry(window,width=40)
    query.grid(column=1, row=1,  padx=5, pady=5)
    
    results = Canvas(window, height=30, width=250)
    results.grid(column=1, row=2, padx=5, pady=5)
    
    # Entry
    def getInputBoxValue():
        userInput = query.get()
        return userInput

    
    # Button
    def clicked():
        search()
        #pass
        
    def no_result():
        tk.messagebox.showwarning("Warning", "No results found. Please try different search terms")
        
    
    def search():
        xtest = Scrapped_hf.copy()
        q = query.get()
        f = Frame(window)
        df = vertical_search_engine(xtest, q, indexed)
        if type(df) == str:
            no_result()
        else:
            pt = Table(results)
            try:
                table = pt = Table(results, dataframe=df)
                pt.show()
            except AttributeError:
                pass
            
    def close_window():
        if tk.messagebox.askokcancel("Quit", "Quit Programme?"):
            window.destroy()
        
    
    btn = Button(window, text="Search", command=clicked)
    btn.grid(column=2, row=1)
    
    

    window.protocol("WM_DELETE_WINDOW", close_window)       
    window.mainloop()

In [58]:
new_gui(image1)